In [1]:
import os
from osgeo import gdal
import numpy as np
import pandas as pd
from datetime import datetime
from tqdm import tqdm

## **Extract AWS data**
* Extract all values
* Filter always-null pixels

In [2]:
import os
from datetime import datetime, timedelta

def extract_timestamp_from_filename(filename):
    """
    Trích xuất timestamp từ tên file theo định dạng 'AWS_YYYYMMDDHHMMSS.tif'
    """
    try:
        base_name = filename.split('.')[0]  # Bỏ phần mở rộng .tif
        parts = base_name.split('_')
        if len(parts) < 2:
            return None
        timestamp_str = parts[-1]  # Phần YYYYMMDDHHMMSS
        timestamp = datetime.strptime(timestamp_str, "%Y%m%d%H%M%S")
        return timestamp
    except Exception:
        return None

def check_missing_hours(root_dir):
    """
    Kiểm tra xem trong tháng 4 và tháng 10 của năm 2019 & 2020 có thiếu bất kỳ giờ nào không.
    Trả về tập hợp (set) các timestamp bị thiếu.
    """
    missing_hours = set()

    for year in ['2019', '2020']:
        for month in ['04', '10']:  # Chỉ kiểm tra tháng 4 và tháng 10
            month_path = os.path.join(root_dir, year, month)
            if not os.path.isdir(month_path):
                print(f"⚠️ Không tìm thấy thư mục tháng: {year}/{month}")
                continue

            expected_hours_per_day = {datetime(int(year), int(month), day, hour) 
                                      for day in range(1, 31)  # Tháng 4 và tháng 10 có 30 ngày
                                      for hour in range(24)}
            available_timestamps = set()

            for day in sorted(os.listdir(month_path)):
                day_path = os.path.join(month_path, day)
                if not os.path.isdir(day_path):
                    continue

                for file in sorted(os.listdir(day_path)):
                    if file.endswith(".tif"):
                        timestamp = extract_timestamp_from_filename(file)
                        if timestamp:
                            available_timestamps.add(timestamp)

            # Xác định timestamp bị thiếu và thêm vào tập `missing_hours`
            missing_hours.update(expected_hours_per_day - available_timestamps)

    if missing_hours:
        print(f"⚠️ Tổng số timestamp bị thiếu: {len(missing_hours)}")
    else:
        print("✅ Dữ liệu đầy đủ, không có giờ nào bị thiếu!")

    return missing_hours if missing_hours else None


In [3]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from osgeo import gdal
from datetime import datetime, timedelta

def extract_data(root_dir, option=1, coordinates=None):
    """
    Trích xuất dữ liệu từ các file GeoTIFF trong thư mục root_dir và bổ sung dữ liệu bị thiếu.
    
    - root_dir: đường dẫn thư mục gốc
    - option: 
        + 1 - Trả về toàn bộ dữ liệu
        + 2 - Chỉ lấy dữ liệu ở các tọa độ cụ thể (row, col) trong tập `coordinates`
    - coordinates: set chứa các tọa độ (row, col) nếu option = 2
    
    Returns:
    - dict chứa DataFrame của từng tháng, với key là "YYYY-MM".
    """
    
    monthly_data = {}
    param_name = root_dir.split('/')[-1]  # Tự động lấy tên biến khí tượng

    for year in ['2019', '2020']:
        year_path = os.path.join(root_dir, year)
        if not os.path.isdir(year_path):
            continue

        for month in sorted(os.listdir(year_path)):
            month_path = os.path.join(year_path, month)
            if not os.path.isdir(month_path):
                continue

            available_timestamps = set()
            pixel_values = {}  # Lưu dữ liệu của từng pixel

            for day in tqdm(sorted(os.listdir(month_path)), desc=f"Đang xử lý {year}/{month}"):
                day_path = os.path.join(month_path, day)
                if not os.path.isdir(day_path):
                    continue

                for file in sorted(os.listdir(day_path)):
                    if not file.endswith(".tif"):
                        continue
                    file_path = os.path.join(day_path, file)

                    dataset = gdal.Open(file_path)
                    if dataset is None:
                        print(f"⚠️ Không mở được file: {file_path}")
                        continue

                    band = dataset.GetRasterBand(1)
                    data = band.ReadAsArray()

                    base = file.split('.')[0]
                    parts = base.split('_')
                    if len(parts) >= 2:
                        dt_str = parts[-1]
                        dt = datetime.strptime(dt_str, "%Y%m%d%H%M%S")
                    else:
                        dt = None
                    
                    available_timestamps.add(dt)

                    rows, cols = data.shape
                    row_idx, col_idx = np.indices((rows, cols))

                    # Áp dụng điều kiện lọc dữ liệu hợp lệ
                    valid_mask = np.ones_like(data, dtype=bool)

                    # Nếu option == 2, chỉ giữ lại các tọa độ có trong `coordinates`
                    if option == 2 and coordinates:
                        coord_mask = np.vectorize(lambda r, c: (r, c) in coordinates)(row_idx, col_idx)
                        valid_mask &= coord_mask

                    values = data[valid_mask]
                    row_idx = row_idx[valid_mask]
                    col_idx = col_idx[valid_mask]

                    for r, c, v in zip(row_idx, col_idx, values):
                        if (r, c) not in pixel_values:
                            pixel_values[(r, c)] = {}
                        pixel_values[(r, c)][dt] = v

            # Tạo danh sách timestamp đầy đủ cho tháng đó
            start_date = datetime(int(year), int(month), 1, 0, 0, 0)
            end_date = datetime(int(year), int(month), 30, 23, 0, 0)
            all_timestamps = {start_date + timedelta(hours=i) for i in range((end_date - start_date).days * 24 + 24)}

            missing_timestamps = all_timestamps - available_timestamps

            # Điền NaN vào các timestamp bị thiếu
            for pixel, values in pixel_values.items():
                for ts in missing_timestamps:
                    values[ts] = np.nan

            # Chuyển dữ liệu thành DataFrame
            data_list = []
            for (r, c), values in pixel_values.items():
                for ts, v in values.items():
                    data_list.append([ts, r, c, v])

            month_key = f"{year}-{month}"
            df = pd.DataFrame(data_list, columns=["datetime", "row", "col", param_name])
            df["datetime"] = pd.to_datetime(df["datetime"])
            df["row"] = df["row"].astype(int)
            df["col"] = df["col"].astype(int)
            df[param_name] = df[param_name].astype(float)

            monthly_data[month_key] = df

    return monthly_data


**Kiểm tra số lượng ảnh từng tháng xem có giống nhau**

In [4]:
import os

def count_tif_files(root_dir):
    """
    Đếm số lượng file .tif trong từng thư mục tháng của root_dir.
    
    Returns:
    - Dictionary { "YYYY-MM": số lượng file .tif }
    """
    file_counts = {}

    for year in ['2019', '2020']:
        year_path = os.path.join(root_dir, year)
        if not os.path.isdir(year_path):
            continue

        for month in sorted(os.listdir(year_path)):
            month_path = os.path.join(year_path, month)
            if not os.path.isdir(month_path):
                continue

            tif_count = sum(
                1 for day in os.listdir(month_path)
                if os.path.isdir(os.path.join(month_path, day))
                for file in os.listdir(os.path.join(month_path, day))
                if file.endswith(".tif")
            )

            file_counts[f"{year}-{month}"] = tif_count

    return file_counts


# Đường dẫn thư mục gốc chứa dữ liệu
root_directory = "/kaggle/input/rainfall-forecast/DATA_SV/Precipitation/AWS"

# Gọi hàm và in kết quả
tif_file_counts = count_tif_files(root_directory)
for month, count in tif_file_counts.items():
    print(f"{month}: {count} files")


2019-04: 720 files
2019-10: 627 files
2020-04: 718 files
2020-10: 742 files


Không giống nhau => có các mốc thời gian bị thiếu tuy nhiên số lượng không đáng kể


In [5]:
# AWS raw data
aws_raw_data = extract_data('/kaggle/input/rainfall-forecast/DATA_SV/Precipitation/AWS')

aws_raw_data.keys()

Đang xử lý 2020/10: 100%|██████████| 31/31 [00:27<00:00,  1.14it/s]


dict_keys(['2019-04', '2019-10', '2020-04', '2020-10'])

In [6]:
def get_valid_pixels(result):
    """
    Xác định tập hợp các pixel (row, col) có ít nhất một giá trị hợp lệ trong bất kỳ tháng nào.
    
    Parameters:
    - result (dict): Dictionary chứa các DataFrame của từng tháng (output từ `extract_data`).
    
    Returns:
    - set: Tập hợp tọa độ (row, col) hợp lệ.
    """
    if not result:
        return set()

    # Lấy tên biến khí tượng từ một DataFrame bất kỳ
    sample_df = next(iter(result.values()))
    param_name = sample_df.columns[-1]  # Cột cuối cùng là giá trị khí tượng (U250, AWS, ...)

    valid_pixels = set()

    # Duyệt qua từng DataFrame để tìm các tọa độ có ít nhất một giá trị hợp lệ
    for df in tqdm(result.values()):
        for row, col, value in zip(df["row"], df["col"], df[param_name]):
            if value != -np.inf and not pd.isna(value):  # Nếu giá trị không phải -inf, thêm vào tập hợp
                valid_pixels.add((row, col))

    return valid_pixels


In [7]:
aws_valid_pixels = get_valid_pixels(aws_raw_data)

len(aws_valid_pixels)

100%|██████████| 4/4 [00:25<00:00,  6.47s/it]


334

In [8]:
filtered_aws_data = extract_data('/kaggle/input/rainfall-forecast/DATA_SV/Precipitation/AWS', option = 2, coordinates=aws_valid_pixels)

Đang xử lý 2020/10: 100%|██████████| 31/31 [00:05<00:00,  5.74it/s]


In [9]:
for key, value in filtered_aws_data.items():
    print(key)
    print(value.info())

2019-04
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240480 entries, 0 to 240479
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   datetime  240480 non-null  datetime64[ns]
 1   row       240480 non-null  int64         
 2   col       240480 non-null  int64         
 3   AWS       240480 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 7.3 MB
None
2019-10
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248496 entries, 0 to 248495
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   datetime  248496 non-null  datetime64[ns]
 1   row       248496 non-null  int64         
 2   col       248496 non-null  int64         
 3   AWS       209418 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 7.6 MB
None
2020-04
<class 'pandas.core.frame.DataFrame'>
RangeI

## Extract ERA5 data
* Extract ERA5 features corresponding to AWS pixels
* Merge into 1 dataframe and save

In [ ]:
import os

# Thư mục chứa dữ liệu ERA5
era5_root = "/kaggle/input/rainfall-forecast/DATA_SV/ERA5"

# Tạo một dictionary để lưu DataFrame của từng thư mục con trong ERA5
era5_data_dict = {}

# Duyệt qua tất cả các thư mục con của ERA5
for subdir in sorted(os.listdir(era5_root)):
    subdir_path = os.path.join(era5_root, subdir)
    
    if os.path.isdir(subdir_path):  # Chỉ xử lý nếu là thư mục
        print(f"📂 Đang xử lý folder con: {subdir}")
        era5_data_dict[subdir] = extract_data(subdir_path, option=2, coordinates=aws_valid_pixels)


In [12]:
output_dir = "/kaggle/working/merged_data"

# Tạo thư mục lưu kết quả
os.makedirs(output_dir, exist_ok=True)

# 🟢 Gộp và lưu từng tháng riêng biệt
for month in filtered_aws_data.keys():
    df = filtered_aws_data[month]  # Lấy DataFrame AWS của tháng đó

    # Merge với từng trường của ERA5 nếu có dữ liệu cùng tháng
    for var_name, era5_dict in era5_data_dict.items():
        if month in era5_dict:
            df = df.merge(era5_dict[month], on=["datetime", "row", "col"], how="left", suffixes=("", f"_{var_name}"))

    # 📝 Lưu DataFrame từng tháng
    output_path = os.path.join(output_dir, f"merged_{month}.csv")
    df.to_csv(output_path, index=False)
    print(f"✅ Đã lưu {output_path}")

print("🎯 Hoàn tất quá trình gộp và lưu!")


✅ Đã lưu /kaggle/working/merged_data/merged_2019-04.csv
✅ Đã lưu /kaggle/working/merged_data/merged_2019-10.csv
✅ Đã lưu /kaggle/working/merged_data/merged_2020-04.csv
✅ Đã lưu /kaggle/working/merged_data/merged_2020-10.csv
🎯 Hoàn tất quá trình gộp và lưu!
